In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import StratifiedKFold
import optuna

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [3]:
train = pd.read_csv(r'./input/playground-series-s3e23/train.csv', index_col = 'id')
test = pd.read_csv(r'./input/playground-series-s3e23/test.csv', index_col = 'id')
orig_train = pd.read_csv(r'./input/software-defect-prediction/jm1.csv')

In [4]:
for object_features in list(orig_train.loc[:, orig_train.dtypes == 'O']):
    orig_train[object_features] = orig_train[object_features].replace({'?' : np.nan}).astype('float64')

In [5]:
X = pd.concat([train, orig_train])
y = X.pop('defects')

seed = 42
splits = 5
skf = StratifiedKFold(n_splits = splits, random_state = seed, shuffle = True)
np.random.seed(seed)

In [6]:
log_transformer = FunctionTransformer(func=np.log1p, validate=False)

In [7]:
from sklearn.model_selection import cross_val_score

def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 10, 500),
        'n_jobs': -1,
        'random_state': seed,
        'max_depth': trial.suggest_int('max_depth', 4, 50),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 150),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 60),
        'criterion': trial.suggest_categorical("criterion", ["gini", "entropy", "log_loss"]),
        'max_features': trial.suggest_categorical('max_features', ["sqrt", "log2"]),
    }

    pipeline = make_pipeline(SimpleImputer(), log_transformer, ExtraTreesClassifier(**params))
    scores = cross_val_score(pipeline, X, y, cv=3, scoring='roc_auc', n_jobs=-1)
    return scores.mean()

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=15, show_progress_bar=True)

[I 2023-10-12 16:51:34,190] A new study created in memory with name: no-name-af626da0-b2da-4284-8f4b-2d7539ec5e57
Best trial: 0. Best value: 0.775981:   7%|██████████▉                                                                                                                                                         | 1/15 [00:02<00:41,  2.93s/it]

[I 2023-10-12 16:51:37,122] Trial 0 finished with value: 0.7759806654108612 and parameters: {'n_estimators': 18, 'max_depth': 24, 'min_samples_split': 30, 'min_samples_leaf': 9, 'criterion': 'log_loss', 'max_features': 'sqrt'}. Best is trial 0 with value: 0.7759806654108612.


Best trial: 1. Best value: 0.777835:  13%|█████████████████████▊                                                                                                                                              | 2/15 [00:10<01:16,  5.89s/it]

[I 2023-10-12 16:51:45,089] Trial 1 finished with value: 0.7778353630301144 and parameters: {'n_estimators': 137, 'max_depth': 23, 'min_samples_split': 31, 'min_samples_leaf': 24, 'criterion': 'gini', 'max_features': 'sqrt'}. Best is trial 1 with value: 0.7778353630301144.


Best trial: 2. Best value: 0.777879:  20%|████████████████████████████████▊                                                                                                                                   | 3/15 [00:17<01:16,  6.37s/it]

...

[I 2023-10-12 16:53:47,555] Trial 12 finished with value: 0.7818807576268298 and parameters: {'n_estimators': 329, 'max_depth': 34, 'min_samples_split': 87, 'min_samples_leaf': 1, 'criterion': 'log_loss', 'max_features': 'log2'}. Best is trial 12 with value: 0.7818807576268298.


Best trial: 12. Best value: 0.781881:  93%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏          | 14/15 [02:24<00:13, 13.23s/it]

[I 2023-10-12 16:53:58,760] Trial 13 finished with value: 0.779750927329914 and parameters: {'n_estimators': 186, 'max_depth': 50, 'min_samples_split': 92, 'min_samples_leaf': 4, 'criterion': 'log_loss', 'max_features': 'log2'}. Best is trial 12 with value: 0.7818807576268298.


Best trial: 12. Best value: 0.781881: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [02:39<00:00, 10.64s/it]

[I 2023-10-12 16:54:13,746] Trial 14 finished with value: 0.7818459509140733 and parameters: {'n_estimators': 228, 'max_depth': 32, 'min_samples_split': 90, 'min_samples_leaf': 1, 'criterion': 'log_loss', 'max_features': 'log2'}. Best is trial 12 with value: 0.7818807576268298.


In [10]:
study.best_params, study.best_value

({'n_estimators': 329,
  'max_depth': 34,
  'min_samples_split': 87,
  'min_samples_leaf': 1,
  'criterion': 'log_loss',
  'max_features': 'log2'},
 0.7818807576268298)

In [9]:
pipeline = make_pipeline(SimpleImputer(), log_transformer, ExtraTreesClassifier(**study.best_params))
pipeline.fit(X, y)

Pipeline(steps=[('simpleimputer', SimpleImputer()),
                ('functiontransformer',
                 FunctionTransformer(func=<ufunc 'log1p'>)),
                ('extratreesclassifier',
                 ExtraTreesClassifier(criterion='log_loss', max_depth=34,
                                      max_features='log2', min_samples_split=87,
                                      n_estimators=329))])

In [11]:
submission = test.copy()
submission['defects'] = pipeline.predict_proba(submission)[:, 1]
submission.defects.to_csv('extra_tree_submission.csv')